In [56]:
from rapidfuzz import process, fuzz
import pandas as pd
import re
import numpy as np

In [204]:
#prepare loan dataset
MD_CD_raw = pd.read_excel("../../Data/Post1790/MD/MD_post1790_CD.xlsx",
                      header = 11, usecols = 'G, H, I, J, K, L, M, U, V, W, X, Y, Z, AA, AI, AJ, AK, AL, AM, AN, AO')
MD_CD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                 'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                 'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
MD_CD_raw['Name'] = MD_CD_raw['First Name'].apply(lambda x: "" if pd.isnull(x) else x) + " " + MD_CD_raw['Last Name'].apply(lambda x: "" if pd.isnull(x) else x)
MD_CD_raw['Name'] = MD_CD_raw['Name'].apply(lambda x: re.sub(' +', ' ', x).strip())
MD_CD_raw['occupation1'] = MD_CD_raw['occupation1'].apply(lambda x: "" if pd.isnull(x) else x)

In [205]:
grouped_df = MD_CD_raw.groupby(['Name', 'town1', 'occupation1']).agg({'6p_Dollar': ['sum', 'count']}).reset_index()
grouped_df.columns = ['Name', 'Town', 'occupation', '6p_total', '6p_count']
grouped_df = grouped_df.sort_values('6p_total', ascending = False).reset_index(drop = True)
grouped_df['pct_cum'] = grouped_df['6p_total'].cumsum()/grouped_df['6p_total'].sum()
grouped_df['pct'] = grouped_df['6p_total']/grouped_df['6p_total'].sum()

In [206]:
grouped_df

,Name,Town,occupation,6p_total,6p_count,pct_cum,pct
0,Benjamin Harwood,Annapolis,,419989.0,7,0.458947,0.458947
1,William Marbury,Annapolis,,55986.0,39,0.520126,0.061179
2,Uriah Forrest and Benjamin Stoddert,George Town,Merchants,29873.0,2,0.552770,0.032644
3,Wallace and Muir,Annapolis,,27925.0,16,0.583286,0.030515
4,James Williams,Annapolis,,23680.0,9,0.609162,0.025877
...,...,...,...,...,...,...,...
277,Ann Rigby,London,,8.0,1,0.999979,0.000009
278,Charles Maccubbin,Annapolis,,6.0,1,0.999986,0.000007
279,Elizabeth Weatherby,Baltimore County,,5.0,1,0.999991,0.000005
280,Andrew Baille,Charles County,,5.0,1,0.999997,0.000005


In [91]:
loan = pd.read_csv('../../Data/Pre1790/cleaned/loan_office_certificates_9_states_cleaned.csv', index_col = 0)
marine = pd.read_csv('../../Data/Pre1790/cleaned/Marine_Liquidated_Debt_Certificates_cleaned.csv', index_col = 0)
pierce = pd.read_csv('../../Data/Pre1790/cleaned/Pierce_Certs_cleaned_2021.csv', index_col = 0)
liquidCT = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_CT_cleaned.csv', index_col = 0)
liquidDE = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_DE_cleaned.csv', index_col = 0)
liquidMA = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_MA_cleaned.csv', index_col = 0)
liquidNJ = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_NJ_cleaned.csv', index_col = 0)
liquidNH = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_NH_cleaned.csv', index_col = 0)
liquidNY = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_NY_cleaned.csv', index_col = 0)
liquidPA1 = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_PA_story_cleaned.csv', index_col = 0)
liquidRI = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_RI_cleaned.csv', index_col = 0)
liquidPA2 = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_PA_stelle_cleaned.csv', index_col = 0)

/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_53970/3372110578.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  pierce = pd.read_csv('../../Data/Pre1790/cleaned/Pierce_Certs_cleaned_2021.csv', index_col = 0)


In [117]:
def nameLiquidated(fname, lname):
    ctsum = liquidCT[[ln in lname and fn in fname for ln, fn in zip(liquidCT['Last name'],
                                                                    liquidCT['First name'])]]['Dollars'].sum()
    desum = liquidDE[[ln in lname and fn in fname for ln, fn in zip(liquidDE['Last name'],
                                                                    liquidDE['First name'])]]['Dollars'].sum()
    masum = liquidMA[[ln in lname and fn in fname for ln, fn in zip(liquidMA['Last name'],
                                                                    liquidMA['First name'])]]['Dollars'].sum()
    njsum = liquidNJ[[ln in lname and fn in fname for ln, fn in zip(liquidNJ['Last name'],
                                                                    liquidNJ['First name'])]]['Dollars'].sum()
    nhsum = liquidNH[[ln in lname and fn in fname for ln, fn in zip(liquidNH['Last name'],
                                                                    liquidNH['First name'])]]['Dollars'].sum()
    pasum1 = liquidPA1[[ln in lname and fn in fname for ln, fn in zip(liquidPA1['Last name'],
                                                                      liquidPA1['First name'])]]['Dollars'].sum()
    risum = liquidRI[[ln in lname and fn in fname for ln, fn in zip(liquidRI['Last name'],
                                                                    liquidRI['First name'])]]['Dollars'].sum()
    pasum2 = liquidPA2[[ln in lname and fn in fname for ln, fn in zip(liquidPA2['Last name'],
                                                                      liquidPA2['First name'])]]['Dollars'].sum()
    return ctsum + desum + masum + njsum + nhsum + pasum1 + risum + pasum2

## Benjamin Harwood

In [95]:
pierce[[ln == 'Harwood' and fn in ['Benjamin', 'Thomas'] for ln, fn in zip(pierce['Last'],
                                                                           pierce['First'])]]['Value'].sum()

877.8800000000001

In [101]:
marine[[ln == 'Harwood' and fn in ['Benjamin', 'Thomas'] for ln, fn in zip(marine['Last name'],
                                                                           marine['First name'])]]['Dollars'].sum()

0.0

In [102]:
loan[[ln == 'Harwood' and fn in ['Benjamin', 'Thomas'] for ln, fn in zip(loan['Last Name 1 '],
                                                                        loan['First Name 1 '])]]['Face Value'].sum()

2400

In [116]:
nameLiquidated(['Harwood'], ['Benjamin', 'Thomas'])

0.0

## William Marbury

In [118]:
pierce[[ln == 'Marbury' and fn in ['William'] for ln, fn in zip(pierce['Last'],
                                                                pierce['First'])]]['Value'].sum()

0.0

In [119]:
marine[[ln == 'Marbury' and fn in ['William'] for ln, fn in zip(marine['Last name'],
                                                                marine['First name'])]]['Dollars'].sum()

0.0

In [120]:
loan[[ln == 'Marbury' and fn in ['William'] for ln, fn in zip(loan['Last Name 1 '],
                                                              loan['First Name 1 '])]]['Face Value'].sum()

0

In [153]:
nameLiquidated(['William'], ['Marbury'])

0.0

## Uriah Forrest and Benjamin Stoddert

In [155]:
marine[[ln in ['Stoddert', 'Forrest'] and fn in ['Uriah', 'Benjamin'] for ln, fn in zip(marine['Last name'],
                                                                                        marine['First name'])]]['Dollars'].sum()

0.0

In [151]:
pierce[[ln in ['Stoddert', 'Forrest'] and fn in ['Uriah', 'Benjamin'] for ln, fn in zip(pierce['Last'],
                                                                                        pierce['First'])]]['Value'].sum()

426.15999999999997

In [148]:
loan[[ln in ['Stoddert', 'Forrest'] and fn in ['Uriah', 'Benjamin'] for ln, fn in zip(loan['Last Name 1 '],
                                                                                      loan['First Name 1 '])]]['Face Value'].sum()

3200

In [154]:
nameLiquidated(['Uriah', 'Benjamin'], ['Stoddert', 'Forrest'])

0.0

## Wallace and Muir

In [182]:
marine[[ln == 'Muir' and fn == 'John' or ln == 'Wallace' and fn == 'Charles'  for ln, fn in zip(marine['Last name'],
                                                                                                marine['First name'])]]['Dollars'].sum()

0.0

In [181]:
pierce[[ln == 'Muir' and fn == 'John' or ln == 'Wallace' and fn == 'Charles'  for ln, fn in zip(pierce['Last'],
                                                                                                pierce['First'])]]['Value'].sum()

28.52

In [170]:
loan[[ln == 'Muir' and fn == 'John' or ln == 'Wallace' and fn == 'Charles'  for ln, fn in zip(loan['Last Name 1 '],
                                                                                              loan['First Name 1 '])]]['Face Value'].sum()

15300

In [175]:
nameLiquidated(['John', 'Charles'], ['Muir', 'Wallace'])

0.0

## James Williams

In [208]:
pierce[[ln == 'Williams' and fn in ['James'] and 'Maryland' in issue for ln, fn, issue in zip(pierce['Last'],
                                                                                              pierce['First'],
                                                                                              pierce['To Whom Issued'])]]['Value'].sum()

101.3

In [185]:
marine[[ln == 'Williams' and fn in ['James'] for ln, fn in zip(marine['Last name'],
                                                                marine['First name'])]]['Dollars'].sum()

0.0

In [188]:
loan[[ln == 'Williams' and fn in ['James'] for ln, fn in zip(loan['Last Name 1 '],
                                                             loan['First Name 1 '])]]['Face Value'].sum()

2300

In [183]:
nameLiquidated(['James'], ['Williams'])

44.0

## John Laird

In [209]:
pierce[[ln == 'Laird' and fn in ['John'] and 'Maryland' in issue for ln, fn, issue in zip(pierce['Last'],
                                                                                          pierce['First'],
                                                                                          pierce['To Whom Issued'])]]['Value'].sum()

0.0

In [210]:
marine[[ln == 'Laird' and fn in ['John'] for ln, fn in zip(marine['Last name'],
                                                           marine['First name'])]]['Dollars'].sum()

0.0

In [211]:
loan[[ln == 'Laird' and fn in ['John'] for ln, fn in zip(loan['Last Name 1 '],
                                                         loan['First Name 1 '])]]['Face Value'].sum()

0

In [212]:
nameLiquidated(['John'], ['Laird'])

0.0

## John Mason

In [222]:
pierce[[ln == 'Mason' and fn in ['John'] and 'Maryland' in issue for ln, fn, issue in zip(pierce['Last'],
                                                                  pierce['First'],
                                                                  pierce['To Whom Issued'])]]['Value'].sum()

0.0

In [216]:
marine[[ln == 'Mason' and fn in ['John'] for ln, fn in zip(marine['Last name'],
                                                           marine['First name'])]]['Dollars'].sum()

0.0

In [219]:
loan[[ln == 'Mason' and fn in ['John'] and state == 8 for ln, fn, state in zip(loan['Last Name 1 '],
                                                                               loan['First Name 1 '],
                                                                               loan['State'])]]

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text,match eligible


In [223]:
nameLiquidated(['John'], ['Mason'])

0.0

## William Campbell

In [228]:
pierce[[ln == 'Campbell' and fn in ['William'] for ln, fn, issue in zip(pierce['Last'],
                                                                  pierce['First'],
                                                                  pierce['To Whom Issued'])]]

,CN,Last,First,Value,Group,To Whom Issued,State,Officer,Last 2,First 2,original text
12794,33475.0,Campbell,William,40.00,47.0,Regiment paid to August 1 1780,NaN,0.0,NaN,NaN,NaN
12795,36337.0,Campbell,William,26.38,54.0,Regiment paid to January 1 1783,NaN,0.0,NaN,NaN,NaN
12796,49258.0,Campbell,William,90.78,67.0,Second Artillery under Col. John Crane Massa...,MA,0.0,NaN,NaN,NaN
12797,49955.0,Campbell,William,108.00,67.0,Second Artillery under Col. John Crane Massa...,MA,0.0,NaN,NaN,NaN
12798,50691.0,Campbell,William,16.47,67.0,Second Artillery under Col. John Crane Massa...,MA,0.0,NaN,NaN,NaN
12799,51175.0,Campbell,William,79.10,67.0,Second Artillery under Col. John Crane Massa...,MA,0.0,NaN,NaN,NaN
12800,52727.0,Campbell,William,1.70,69.0,Eighth Massachusetts Regiment under Col. Mich...,MA,0.0,NaN,NaN,NaN
12801,58673.0,Campbell,William,92.40,74.0,Invalid Corps of United States under Col. L. ...,NaN,0.0,NaN,NaN,NaN
12802,59368.0,Campbell,William,17.86,74.0,Invalid Corps of United States under Col. L. ...,NaN,0.0,NaN,NaN,NaN
12803,76747.0,Campbell,William,142.12,96.0,Pennsylvania Artillery under Lieut. Col. F. P...,PA,0.0,NaN,NaN,NaN


In [216]:
marine[[ln == 'Mason' and fn in ['John'] for ln, fn in zip(marine['Last name'],
                                                           marine['First name'])]]['Dollars'].sum()

0.0

In [219]:
loan[[ln == 'Mason' and fn in ['John'] and state == 8 for ln, fn, state in zip(loan['Last Name 1 '],
                                                                               loan['First Name 1 '],
                                                                               loan['State'])]]

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text,match eligible


In [212]:
nameLiquidated(['John'], ['Laird'])

0.0